In [13]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]

In [11]:
import pandas as pd #для работы с таблицами

# Объедините два массива в DataFrame. 

In [15]:
plen=len(petersburg)
mlen=len(magadan)
from cmath import nan
while plen!=mlen:
    if plen>mlen:
        magadan.append(nan)
        mlen=len(magadan)
    else:
        petersburg.append(nan)
        plen=len(petersburg)
data = pd.DataFrame({
        'peterburg': petersburg,
        'magadan': magadan
        },
    )
display(data)

,peterburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


# Тест на нормальность

Для выбора статистического теста требуется знать нормально распределены признаки или нет, поэтому сначала проведем тест на нормальность

In [16]:
#проверим данные по Санкт-Петербургу
H0 = 'Данные по мидиям из Санкт-Петербурга распределены нормально'
Ha = 'Данные по мидиям из Санкт-Петербурга не распределены нормально (мы отвергаем H0)'
alpha = 0.05

In [18]:
# используем тест Шапиро-Уилка, для Д"Агостино требуется 8 и более значение, он нам не подходит.
from scipy.stats import shapiro
_, p = shapiro(data['peterburg'].dropna())
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.242
Данные по мидиям из Санкт-Петербурга распределены нормально


In [19]:
#проверим данные по Магадану
H0 = 'Данные по мидиям из Магадана распределены нормально'
Ha = 'Данные по мидиям из Магадана не распределены нормально (мы отвергаем H0)'
alpha = 0.05

In [20]:
# используем тест Шапиро-Уилка.
from scipy.stats import shapiro
_, p = shapiro(data['magadan'])
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.036
Данные по мидиям из Магадана не распределены нормально (мы отвергаем H0)


Итого  один набор данных по мидиям из Санкт-Петербурга у нас распределен нормально. второй набор данных по мидиям из Магадана не распределен нормально, следовательно нам требуется использовать непараметрические тесты. так как обе переменные у нас количественные, то можем использовать тест на корреляцию Спирмена

# Тест на корреляцию

In [23]:
#заполним значение nan средним
data['peterburg'] = data['peterburg'].fillna(data['peterburg'].mean())

In [25]:
from scipy.stats import spearmanr
H0 = 'Отсутствует связь между переменными'
Ha = 'Не отсутствует связь между переменными (мы отвергаем H0)'
alpha = 0.05

corr, p = spearmanr(data['peterburg'], data['magadan'])
print(f"Корреляция состявляет {corr} при уровне значимости {p}")
if p>alpha:
    print(H0)
else:
    print(Ha)

Корреляция состявляет 0.1666666666666667 при уровне значимости 0.693238811728395
Отсутствует связь между переменными


Мы не можем отвергнуть нулевую гипотезу, что отсутствует связь между переменными. Так как у нас было 2 разных набора по размеру раковин из разных городов, которые не собирались в одно и то же время, то выглядит логичным, что связи между этими наборами данных не наблюдается.

# Разница в среднем размере мидии в зависимости от города производителя

независимая переменная для данного теста город-производитель, это категориальная переменная, зависимая переменная количественная. 
Ранее мы выяснили, что распределение не соответствует нормальному и нам нужен непараметрический тест. Подойдет тест сумма рангов Уилкоксона, им мы и воспользуемся

In [27]:
from scipy.stats import wilcoxon
H0 = 'Нет значимой разницы между размерами раковин в разных городах.'
Ha = 'Есть значимая разница между размерами раковин в разных городах.'
alpha = 0.05
test_results = wilcoxon(data['peterburg'], data['magadan'])
p = round(test_results[1],2)
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.02 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами раковин в разных городах.


Мы не можем отвергнуть нулевую гипотезу, следовательно есть значимая разница между размерами раковин с разных городах. Следовательно есть разница в среднем размере мидии в зависимости от города-производителя.